In [1]:
import zipfile
import os

def extract_files(zip_path):
    extract_dir = os.path.dirname(zip_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extracted all files")

extract_files("archive.zip")

Extracted all files


In [1]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import torch
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [2]:
class FireNoFireDataset(Dataset):
    def __init__(self, annotations_dir, images_dir, transform=None):
        self.annotations_dir = annotations_dir
        self.images_dir = images_dir
        self.transform = transform
        self.data = self._load_data()

    def _load_data(self):
        data = []
        for xml_file in os.listdir(self.annotations_dir):
            if xml_file.endswith('.xml'):
                try:
                    tree = ET.parse(os.path.join(self.annotations_dir, xml_file))
                    root = tree.getroot()

                    object_tag = root.find('object')
                    if object_tag is not None:
                        name_tag = object_tag.find('name')
                        if name_tag is not None:
                            label_text = name_tag.text.strip()
                            label = 1 if label_text.lower() == 'fire' else 0
                        else:
                            print(f"Warning: <name> tag missing in {xml_file}")
                            continue
                    else:
                        print(f"Warning: <object> tag missing in {xml_file}")
                        continue

                    # for extracting corresponding image file name
                    filename_tag = root.find('filename')
                    if filename_tag is not None:
                        filename = filename_tag.text.strip()
                        img_path = os.path.join(self.images_dir, filename)

                        if os.path.exists(img_path):
                            data.append((img_path, label))
                        else:
                            print(f"Warning: Image file {filename} not found for {xml_file}")
                    else:
                        print(f"Warning: <filename> tag missing in {xml_file}")
                except ET.ParseError:
                    print(f"Error: Failed to parse {xml_file}. Skipping.")
        return data


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


annotations_dir = "../Annotations/Annotations"
images_dir = "../Datacluster Fire and Smoke Sample/Datacluster Fire and Smoke Sample"
dataset = FireNoFireDataset(annotations_dir, images_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [5]:
model = models.resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2) 

model.load_state_dict(torch.load('deepfire_resnet50.pth'))
model.eval()

/local_scratch/slurm.1244978/ipykernel_1640929/2264215474.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('deepfire_resnet50.pth'))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Model Accuracy: {accuracy:.2f}%")

Model Accuracy: 35.35%


In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score

correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        # Append predictions and true labels for metric computation
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        # Update accuracy calculation
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate metrics
accuracy = 100 * correct / total
precision = precision_score(all_labels, all_preds, average='binary')  # Use 'binary' for binary classification
recall = recall_score(all_labels, all_preds, average='binary')
f1 = f1_score(all_labels, all_preds, average='binary')

# Print results
print(f"Model Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Model Accuracy: 35.35%
Precision: 1.00
Recall: 0.35
F1 Score: 0.52


In [1]:
import os

def count_jpg_files(directory):
    # Count files ending with .jpg or .jpeg (case insensitive)
    jpg_count = len([f for f in os.listdir(directory) 
                    if f.lower().endswith(('.jpg', '.jpeg'))])
    return jpg_count

# Example usage
directory = "Datacluster Fire and Smoke Sample/Datacluster Fire and Smoke Sample"
count = count_jpg_files(directory)
print(f"Found {count} JPG files")

Found 100 JPG files
